In [0]:
from pyspark.sql import Row
from pyspark.sql.functions import col, to_timestamp, minute, desc, rank
from pyspark.sql.window import Window

web_data = [
    Row(UserID=1, Page="Home", Timestamp="2024-04-10 10:00:00", Duration=35, Device="Mobile", Country="India"),
    Row(UserID=2, Page="Products", Timestamp="2024-04-10 10:02:00", Duration=120, Device="Desktop", Country="USA"),
    Row(UserID=3, Page="Cart", Timestamp="2024-04-10 10:05:00", Duration=45, Device="Tablet", Country="UK"),
    Row(UserID=1, Page="Checkout", Timestamp="2024-04-10 10:08:00", Duration=60, Device="Mobile", Country="India"),
    Row(UserID=4, Page="Home", Timestamp="2024-04-10 10:10:00", Duration=15, Device="Mobile", Country="Canada"),
    Row(UserID=2, Page="Contact", Timestamp="2024-04-10 10:15:00", Duration=25, Device="Desktop", Country="USA"),
    Row(UserID=5, Page="Products", Timestamp="2024-04-10 10:20:00", Duration=90, Device="Desktop", Country="India"),
]

df_web = spark.createDataFrame(web_data)
df_web.show(truncate=False)


+------+--------+-------------------+--------+-------+-------+
|UserID|Page    |Timestamp          |Duration|Device |Country|
+------+--------+-------------------+--------+-------+-------+
|1     |Home    |2024-04-10 10:00:00|35      |Mobile |India  |
|2     |Products|2024-04-10 10:02:00|120     |Desktop|USA    |
|3     |Cart    |2024-04-10 10:05:00|45      |Tablet |UK     |
|1     |Checkout|2024-04-10 10:08:00|60      |Mobile |India  |
|4     |Home    |2024-04-10 10:10:00|15      |Mobile |Canada |
|2     |Contact |2024-04-10 10:15:00|25      |Desktop|USA    |
|5     |Products|2024-04-10 10:20:00|90      |Desktop|India  |
+------+--------+-------------------+--------+-------+-------+



Data Exploration & Preparation

1. Display the schema of web_traffic_data .

In [0]:
df_web.printSchema()


root
 |-- UserID: long (nullable = true)
 |-- Page: string (nullable = true)
 |-- Timestamp: string (nullable = true)
 |-- Duration: long (nullable = true)
 |-- Device: string (nullable = true)
 |-- Country: string (nullable = true)



2. Convert the Timestamp column to a proper timestamp type.

In [0]:
df_web = df_web.withColumn("Timestamp", to_timestamp("Timestamp"))


3. Add a new column SessionMinute by extracting the minute from the Timestamp .

In [0]:
from pyspark.sql.functions import minute

df_web = df_web.withColumn("SessionMinute", minute("Timestamp"))
df_web.select("UserID", "Page", "SessionMinute").show()


+------+--------+-------------+
|UserID|    Page|SessionMinute|
+------+--------+-------------+
|     1|    Home|            0|
|     2|Products|            2|
|     3|    Cart|            5|
|     1|Checkout|            8|
|     4|    Home|           10|
|     2| Contact|           15|
|     5|Products|           20|
+------+--------+-------------+



Filtering and Conditions

4. Filter users who used a "Mobile" device and visited the "Checkout" page.

In [0]:
df_web.filter((col("Device") == "Mobile") & (col("Page") == "Checkout")).show()


+------+--------+-------------------+--------+------+-------+-------------+
|UserID|    Page|          Timestamp|Duration|Device|Country|SessionMinute|
+------+--------+-------------------+--------+------+-------+-------------+
|     1|Checkout|2024-04-10 10:08:00|      60|Mobile|  India|            8|
+------+--------+-------------------+--------+------+-------+-------------+



5. Show all entries with a Duration greater than 60 seconds.

In [0]:
df_web.filter(col("Duration") > 60).show()


+------+--------+-------------------+--------+-------+-------+-------------+
|UserID|    Page|          Timestamp|Duration| Device|Country|SessionMinute|
+------+--------+-------------------+--------+-------+-------+-------------+
|     2|Products|2024-04-10 10:02:00|     120|Desktop|    USA|            2|
|     5|Products|2024-04-10 10:20:00|      90|Desktop|  India|           20|
+------+--------+-------------------+--------+-------+-------+-------------+



6. Find all users from India who visited the "Products" page.

In [0]:
df_web.filter((col("Country") == "India") & (col("Page") == "Products")).show()


+------+--------+-------------------+--------+-------+-------+-------------+
|UserID|    Page|          Timestamp|Duration| Device|Country|SessionMinute|
+------+--------+-------------------+--------+-------+-------+-------------+
|     5|Products|2024-04-10 10:20:00|      90|Desktop|  India|           20|
+------+--------+-------------------+--------+-------+-------+-------------+



Aggregation and Grouping

7. Get the average duration per device type.

In [0]:
df_web.groupBy("Device").avg("Duration").withColumnRenamed("avg(Duration)", "AvgDuration").show()


+-------+------------------+
| Device|       AvgDuration|
+-------+------------------+
| Mobile|36.666666666666664|
| Tablet|              45.0|
|Desktop| 78.33333333333333|
+-------+------------------+



8. Count the number of sessions per country.

In [0]:
df_web.groupBy("Country").count().withColumnRenamed("count", "SessionCount").show()


+-------+------------+
|Country|SessionCount|
+-------+------------+
|  India|           3|
|    USA|           2|
|     UK|           1|
| Canada|           1|
+-------+------------+



9. Find the most visited page overall.

In [0]:
df_web.groupBy("Page").count().orderBy(desc("count")).limit(1).show()


+----+-----+
|Page|count|
+----+-----+
|Home|    2|
+----+-----+



Window Functions

10. Rank each user’s pages by timestamp

In [0]:
windowSpec = Window.partitionBy("UserID").orderBy("Timestamp")
df_ranked = df_web.withColumn("Rank", rank().over(windowSpec))
df_ranked.select("UserID", "Page", "Timestamp", "Rank").show()


+------+--------+-------------------+----+
|UserID|    Page|          Timestamp|Rank|
+------+--------+-------------------+----+
|     1|    Home|2024-04-10 10:00:00|   1|
|     1|Checkout|2024-04-10 10:08:00|   2|
|     2|Products|2024-04-10 10:02:00|   1|
|     2| Contact|2024-04-10 10:15:00|   2|
|     3|    Cart|2024-04-10 10:05:00|   1|
|     4|    Home|2024-04-10 10:10:00|   1|
|     5|Products|2024-04-10 10:20:00|   1|
+------+--------+-------------------+----+



11. Find the total duration of all sessions per user using groupBy .

In [0]:
df_web.groupBy("UserID").sum("Duration").withColumnRenamed("sum(Duration)", "TotalDuration").show()


+------+-------------+
|UserID|TotalDuration|
+------+-------------+
|     1|           95|
|     3|           45|
|     2|          145|
|     4|           15|
|     5|           90|
+------+-------------+



Spark SQL Tasks

12. Create a temporary view called traffic_view .

In [0]:
df_web.createOrReplaceTempView("traffic_view")


13. Write a SQL query to get the top 2 longest sessions by duration.

In [0]:
spark.sql("""
    SELECT *
    FROM traffic_view
    ORDER BY Duration DESC
    LIMIT 2
""").show()


+------+--------+-------------------+--------+-------+-------+-------------+
|UserID|    Page|          Timestamp|Duration| Device|Country|SessionMinute|
+------+--------+-------------------+--------+-------+-------+-------------+
|     2|Products|2024-04-10 10:02:00|     120|Desktop|    USA|            2|
|     5|Products|2024-04-10 10:20:00|      90|Desktop|  India|           20|
+------+--------+-------------------+--------+-------+-------+-------------+



14. Get the number of unique users per page using SQL.

In [0]:
spark.sql("""
    SELECT Page, COUNT(DISTINCT UserID) as UniqueUsers
    FROM traffic_view
    GROUP BY Page
""").show()


+--------+-----------+
|    Page|UniqueUsers|
+--------+-----------+
|    Cart|          1|
|    Home|          2|
|Checkout|          1|
|Products|          2|
| Contact|          1|
+--------+-----------+



Export & Save

15. Save the final DataFrame to CSV.

In [0]:
df_web.write.mode("overwrite").option("header", True).csv("/tmp/web_traffic_csv")


16. Save partitioned by Country in Parquet format.

In [0]:
df_web.write.mode("overwrite").partitionBy("Country").parquet("/tmp/web_traffic_parquet")
